In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer


In [5]:
import pandas as pd
from datasets import Dataset

# Load your sentiment dataset again (assuming you saved it)
sentiment_df = pd.read_csv("../data/sentiment_dataset.csv")
emotion_df = pd.read_csv("../data/emotion_dataset.csv")

# Convert sentiment DataFrame to Hugging Face Dataset
sentiment_dataset_hf = Dataset.from_pandas(sentiment_df)

# Convert emotion DataFrame to Hugging Face Dataset
emotion_dataset_hf = Dataset.from_pandas(emotion_df)

print(sentiment_dataset_hf)
print(emotion_dataset_hf)




Dataset({
    features: ['text', 'source', 'language', 'label'],
    num_rows: 270399
})
Dataset({
    features: ['text', 'label', 'emotion'],
    num_rows: 16000
})


In [6]:
from datasets import DatasetDict

# Sentiment dataset split
sentiment_dataset_hf = sentiment_dataset_hf.train_test_split(test_size=0.2, seed=42)
sentiment_valid_test = sentiment_dataset_hf['test'].train_test_split(test_size=0.5, seed=42)
sentiment_datasets = DatasetDict({
    'train': sentiment_dataset_hf['train'],
    'validation': sentiment_valid_test['train'],
    'test': sentiment_valid_test['test']
})

# Emotion dataset split
emotion_dataset_hf = emotion_dataset_hf.train_test_split(test_size=0.2, seed=42)
emotion_valid_test = emotion_dataset_hf['test'].train_test_split(test_size=0.5, seed=42)
emotion_datasets = DatasetDict({
    'train': emotion_dataset_hf['train'],
    'validation': emotion_valid_test['train'],
    'test': emotion_valid_test['test']
})

# Verify the splits
print(sentiment_datasets)
print(emotion_datasets)


DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'language', 'label'],
        num_rows: 216319
    })
    validation: Dataset({
        features: ['text', 'source', 'language', 'label'],
        num_rows: 27040
    })
    test: Dataset({
        features: ['text', 'source', 'language', 'label'],
        num_rows: 27040
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'emotion'],
        num_rows: 12800
    })
    validation: Dataset({
        features: ['text', 'label', 'emotion'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label', 'emotion'],
        num_rows: 1600
    })
})


In [7]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenization function
def tokenize_batch(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

# Tokenize sentiment dataset
tokenized_sentiment = sentiment_datasets.map(tokenize_batch, batched=True)

# Tokenize emotion dataset
tokenized_emotion = emotion_datasets.map(tokenize_batch, batched=True)

# Check a sample tokenized entry
print(tokenized_sentiment['train'][0])
print(tokenized_emotion['train'][0])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\multilingual-transformer-sentiment-emotion-5vIiU82Q-py3.12\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

Map:   0%|          | 0/27040 [00:00<?, ? examples/s]

Map:   0%|          | 0/12800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

{'text': 'ハンズフリーイヤフォン・kindle・スマホ・Bluetoothスピーカーなどを充電するとき、複数のアダプタを使用するのは面倒なので購入しました。同時使用機器数を考慮し３ポートの本機を購入しました。他の同種の商品で折り畳みプラグ方式の耐久性に難ありとのamazonレビューを参考に固定式プラグをチョイスしました。 スマホが急速充電中にはそれなりの電流値が表示されているようです。今のところ快適に使用しています。 プラグの耐久性は心配ないので、これから先、回路が長期間の仕様に耐えてくれることを期待しています。', 'source': 'amazon_reviews_multi', 'language': 'japanese', 'label': 0, 'input_ids': [0, 6, 158676, 13591, 119415, 161996, 100537, 7740, 1925, 68347, 133, 1925, 72040, 1925, 114567, 188, 29827, 3385, 78694, 48275, 45485, 97047, 1786, 32658, 37, 130159, 6242, 27857, 16985, 10793, 187014, 6814, 145192, 15871, 32299, 9575, 30, 16324, 2229, 74086, 4723, 251, 49162, 1373, 363, 177799, 154, 1516, 3752, 156530, 9575, 30, 27329, 3169, 10694, 154, 6689, 507, 16059, 2694, 244417, 6095, 133628, 21300, 6518, 154, 26361, 15954, 1278, 327, 10648, 41792, 18513, 54149, 191, 168788, 215103, 41284, 5392, 133628, 21300, 251, 178334, 94586, 9575, 30, 6, 72040, 281, 222163, 97047, 514, 2880, 24054, 57233, 154, 6312, 4695, 45975, 151271, 7826, 256

In [8]:
tokenized_sentiment.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_emotion.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [9]:
tokenized_sentiment.save_to_disk("../data/tokenized_sentiment")
tokenized_emotion.save_to_disk("../data/tokenized_emotion")


Saving the dataset (0/1 shards):   0%|          | 0/216319 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27040 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27040 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12800 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

In [10]:
print(tokenized_sentiment)
print(tokenized_emotion)


DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'language', 'label', 'input_ids', 'attention_mask'],
        num_rows: 216319
    })
    validation: Dataset({
        features: ['text', 'source', 'language', 'label', 'input_ids', 'attention_mask'],
        num_rows: 27040
    })
    test: Dataset({
        features: ['text', 'source', 'language', 'label', 'input_ids', 'attention_mask'],
        num_rows: 27040
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'emotion', 'input_ids', 'attention_mask'],
        num_rows: 12800
    })
    validation: Dataset({
        features: ['text', 'label', 'emotion', 'input_ids', 'attention_mask'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label', 'emotion', 'input_ids', 'attention_mask'],
        num_rows: 1600
    })
})
